In [1]:
print(1+2)

3


In [2]:
import sys
print(sys.executable)
print(sys.version)
import qiskit
print(qiskit.__version__)


/Users/shyam/Documents/Quantum Projects/vqe_env/bin/python
3.11.14 (main, Oct  9 2025, 16:16:55) [Clang 17.0.0 (clang-1700.0.13.3)]
0.45.3


In [3]:
import qiskit
import qiskit_nature
import qiskit_aer
import pyscf

print(qiskit.__version__)
print(qiskit_nature.__version__)
print(qiskit_aer.__version__)
print(pyscf.__version__)


0.45.3
0.6.2
0.12.2
2.11.0


In [4]:
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.algorithms import VQEUCCFactory

In [5]:
from qiskit_aer import Aer
from qiskit.primitives import Estimator

In [6]:
# -------------------------------------------------------
# 1. Build H2 molecule at 0.735 angstroms
# -------------------------------------------------------
driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g"
)

problem = driver.run()
print(problem)

In [7]:

# -------------------------------------------------------
# 2. Active space transformer (2 electrons, 2 orbitals)
# -------------------------------------------------------
transformer = ActiveSpaceTransformer(2, 2)
problem = transformer.transform(problem)
print(problem)


In [8]:

# -------------------------------------------------------
# 3. Map to qubit Hamiltonian
# -------------------------------------------------------
mapper = ParityMapper(num_particles=problem.num_particles)

In [15]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_algorithms.optimizers import SLSQP


In [17]:
from qiskit_nature.second_q.circuit.library import UCC


# -------------------------------------------------------
# 4. Build UCCSD ansatz
# -------------------------------------------------------
ansatz = UCC(
    num_spatial_orbitals=problem.num_spatial_orbitals,
    num_particles=problem.num_particles,
    excitations="sd"   # singles + doubles = UCCSD
)


In [18]:
# -------------------------------------------------------
# 5. Setup VQE
# -------------------------------------------------------
optimizer = SLSQP(maxiter=200)
estimator = Estimator()

vqe_solver = VQEUCCFactory(
    estimator=estimator,
    ansatz=ansatz,
    optimizer=optimizer
)

/var/folders/7x/ghch8w1j37z38f_5vr6c_lw00000gn/T/ipykernel_67332/3903338619.py:7: DeprecationWarning: The VQEUCCFactory class is deprecated as of version 0.6.0 and will be removed no sooner than 3 months after the release . This class is deprecated without replacement. Instead, refer to this how-to guide which explains the steps you need to take to replace the use of this class: https://qiskit.org/documentation/nature/howtos/vqe_ucc.html.
  vqe_solver = VQEUCCFactory(


In [19]:
# -------------------------------------------------------
# 6. Ground-state solver
# -------------------------------------------------------
calc = GroundStateEigensolver(mapper, vqe_solver)

In [20]:
# -------------------------------------------------------
# 7. Solve
# -------------------------------------------------------
result = calc.solve(problem)

/Users/shyam/Documents/Quantum Projects/vqe_env/lib/python3.11/site-packages/qiskit_nature/deprecation.py:297: PauliSumOpDeprecationWarning: PauliSumOp is deprecated as of version 0.6.0 and support for them will be removed no sooner than 3 months after the release. Instead, use SparsePauliOp. You can switch to SparsePauliOp immediately, by setting `qiskit_nature.settings.use_pauli_sum_op` to `False`.
  return func(*args, **kwargs)


In [21]:
print("Ground-state energy (Hartree):", result.total_energies[0])

Ground-state energy (Hartree): -1.1373060356955667
